In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [4]:
llm.invoke("Hi do you think I am a good person?")

AIMessage(content='I\'m happy to chat with you, but I want to clarify that I\'m a large language model, I don ”t have personal opinions or have the ability to make judgments about individuals, including whether someone is a "good" person or not. \n\nHowever, I can tell you that being a "good" person is a complex and subjective concept that can be influenced by many factors, including your values, actions, and how you treat others. It\'s also important to remember that everyone makes mistakes and has flaws, and it\'s how we learn and grow from those mistakes that matters.\n\nWhat\'s more important is what you think about yourself and how you strive to be a good person in your own eyes. What do you think makes a good person, and how do you strive to embody those qualities in your own life?\n\nI\'m happy to listen and offer support if you\'d like to discuss your thoughts and feelings. How are you feeling today, and is there anything on your mind that\'s making you wonder about this?', add

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, END, StateGraph